Hay dos modelos implementados:

**1** - *Predicciones con MLPRegressor*

**2** - *Predicciones con RandomForest*

Ambos modelos dan predicciones muy distintas y lo más importante es que el dataset es muy malo. 
A lo mejor, en partidas de torneos en las que tienes que jugar horas, la media de dinero que apuestas tras
jugar en todas las rondas está en torno al 10-35 por ciento de tu dinero actual (Salvo partidas extremadamente agresivas).
Pues como en este dataset, los jugadores siempre empiezan con el mismo dinero, los jugadores son muy agresivos,
lo que hace que se estimen valores muy altos.

Además, como no hay muchos datos, el resultado varía extremadamente si se usa un 80, 90 o 99% de los datos 
(hay una parte que hay que dejar para el test aunque no usemos). Lo que hace que sea completamente random el resultado final.
A lo mejor usando un 90% de datos estima un 60% del dinero original y con un  99% se estima un 85%.

Conclusión: o aplicamos una suerte de normalización de los datos para bajar las apuestas que se hagan, o usamos los datos así,
o hacemos un cherrypicking de datos que nos gusten.

In [28]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

class NeuralNetwork:
    def __init__(self):
        self.data = pd.read_csv('datasets/cleaned_data.csv')
        
        # Modelos para cada etapa
        self.preflop_mean = self.data['total'].mean()  # Como no tenemos datos, nos quedaremos con la media de lo que se suele apostar.
        self.flop_predictor = None
        self.turn_predictor = None
        self.river_predictor = None
        
        self.load_data()
    
    
    def load_data(self):
        for stage in ['preflop', 'flop', 'turn', 'river']:
            self.data[[f'{stage}_player', f'{stage}_total']] = pd.DataFrame(self.data[stage].apply(eval).tolist(), index=self.data.index)
        self.data.drop(columns=['preflop', 'flop', 'turn', 'river'], inplace=True)

    
    def train_models(self) -> None:
        # Entrenar modelos
        self._train_flop_model()
        self._train_turn_model()
        self._train_river_model()
    
    
    def _train_flop_model(self) -> None:
        X = self.data[['preflop_total', 'preflop_player']]
        y = self.data['total']
        
        # Dividir datos
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=33)
        
        # Escalar características
        self.flop_scaler = StandardScaler()
        X_train_scaled = self.flop_scaler.fit_transform(X_train)
        
        # Entrenar modelo
        self.flop_predictor = MLPRegressor(
            hidden_layer_sizes=(10, 5), 
            max_iter=1000, 
            random_state=33
        )
        self.flop_predictor.fit(X_train_scaled, y_train)
    
    
    def _train_turn_model(self) -> None:
        # Preparar características y objetivo
        X = self.data[['preflop_total', 'preflop_player', 'flop_total', 'flop_player']]
        y = self.data['total']
        
        # Dividir datos
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=33)
        
        # Escalar características
        self.turn_scaler = StandardScaler()
        X_train_scaled = self.turn_scaler.fit_transform(X_train)
        
        # Entrenar modelo
        self.turn_predictor = MLPRegressor(
            hidden_layer_sizes=(15, 7), 
            max_iter=1000, 
            random_state=33
        )
        self.turn_predictor.fit(X_train_scaled, y_train)
    
    
    def _train_river_model(self) -> None:
        # Preparar características y objetivo
        X = self.data[['preflop_total', 'preflop_player', 'flop_total', 'flop_player', 
                       'turn_total', 'turn_player']]
        y = self.data['total']
        
        # Dividir datos
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=33)
        
        # Escalar características
        self.river_scaler = StandardScaler()
        X_train_scaled = self.river_scaler.fit_transform(X_train)
        
        # Entrenar modelo
        self.river_predictor = MLPRegressor(
            hidden_layer_sizes=(20, 10), 
            max_iter=1000, 
            random_state=33
        )
        self.river_predictor.fit(X_train_scaled, y_train)
    
    
    def save_models(self, path_prefix: str = 'poker_model') -> None:
        # Guardar scaler
        joblib.dump(self.flop_scaler, f'{path_prefix}_flop_scaler.joblib')
        joblib.dump(self.turn_scaler, f'{path_prefix}_turn_scaler.joblib')
        joblib.dump(self.river_scaler, f'{path_prefix}_river_scaler.joblib')
        
        # Guardar modelos
        if self.flop_predictor:
            joblib.dump(self.flop_predictor, f'{path_prefix}_flop.joblib')
        if self.turn_predictor:
            joblib.dump(self.turn_predictor, f'{path_prefix}_turn.joblib')
        if self.river_predictor:
            joblib.dump(self.river_predictor, f'{path_prefix}_river.joblib')
        
        # Guardar media de preflop
        with open(f'{path_prefix}_preflop_mean.txt', 'w') as f:
            f.write(str(self.preflop_mean))
    
    
    def load_models(self, path_prefix: str = 'poker_model') -> None:
        # Cargar scaler
        self.flop_scaler = joblib.load(f'{path_prefix}_flop_scaler.joblib')
        self.turn_scaler = joblib.load(f'{path_prefix}_turn_scaler.joblib')
        self.river_scaler = joblib.load(f'{path_prefix}_river_scaler.joblib')
        self.scaler = joblib.load(f'{path_prefix}_scaler.joblib')
        
        # Cargar modelos
        try:
            self.flop_predictor = joblib.load(f'{path_prefix}_flop.joblib')
            self.turn_predictor = joblib.load(f'{path_prefix}_turn.joblib')
            self.river_predictor = joblib.load(f'{path_prefix}_river.joblib')
        except FileNotFoundError:
            print("Algunos modelos no se encontraron. Asegúrate de haberlos guardado previamente.")
        
        # Cargar media de preflop
        with open(f'{path_prefix}_preflop_mean.txt', 'r') as f:
            self.preflop_mean = float(f.read())
    
    
    def preflop_model(self) -> float:
        if self.preflop_mean is None:
            raise ValueError("La media de preflop no ha sido inicializada. Carga o entrena primero.")
        return self.preflop_mean
    
    
    def flop_model(self, preflop_bet: float) -> float:
        if self.flop_predictor is None:
            raise ValueError("El modelo de flop no ha sido entrenado. Entrena primero.")
        
        # Preparar input para predicción
        input_data = np.array([[preflop_bet, preflop_bet]])
        input_df = pd.DataFrame(input_data, columns=['preflop_total', 'preflop_player'])
        input_scaled = self.flop_scaler.transform(input_df[['preflop_total', 'preflop_player']])
        
        # Predecir
        return float(self.flop_predictor.predict(input_scaled)[0])
    
    
    def turn_model(self, preflop_bet: float, flop_bet: float) -> float:
        if self.turn_predictor is None:
            raise ValueError("El modelo de turn no ha sido entrenado. Entrena primero.")
        
        input_data = np.array([[preflop_bet, preflop_bet, flop_bet, flop_bet]])
        input_df = pd.DataFrame(input_data, columns=['preflop_total', 'preflop_player', 'flop_total', 'flop_player'])
        input_scaled = self.turn_scaler.transform(input_df[['preflop_total', 'preflop_player', 'flop_total', 'flop_player']])
        
        # Predecir
        return float(self.turn_predictor.predict(input_scaled)[0])
    
    
    def river_model(self, preflop_bet: float, flop_bet: float, turn_bet: float) -> float:
        if self.river_predictor is None:
            raise ValueError("El modelo de river no ha sido entrenado. Entrena primero.")
        
        input_data = np.array([[preflop_bet, preflop_bet, flop_bet, flop_bet, turn_bet, turn_bet]])
        input_df = pd.DataFrame(input_data, columns=['preflop_total', 'preflop_player', 'flop_total', 'flop_player', 'turn_total', 'turn_player'])
        input_scaled = self.river_scaler.transform(input_df[['preflop_total', 'preflop_player', 'flop_total', 'flop_player', 'turn_total', 'turn_player']])
        input_data = np.array([[preflop_bet, preflop_bet, flop_bet, flop_bet, turn_bet, turn_bet]])
        input_scaled = self.river_scaler.transform(input_data)
        
        # Predecir
        return float(self.river_predictor.predict(input_scaled)[0])
    
    
# Prueba 1
nn = NeuralNetwork()
nn.train_models()

# Predicciones
preflop_bet = 0.05 
flop_bet = 0.1  
turn_bet = 0.2  

preflop_prediction = nn.preflop_model()
flop_prediction = nn.flop_model(preflop_bet)
turn_prediction = nn.turn_model(preflop_bet, flop_bet)
river_prediction = nn.river_model(preflop_bet, flop_bet, turn_bet)

print(f"Predicción de preflop: {preflop_prediction}")
print(f"Predicción de flop: {flop_prediction}")
print(f"Predicción de turn: {turn_prediction}")
print(f"Predicción de river: {river_prediction}\n")



# Prueba tras guardar el modelo
nn.save_models()

nn = NeuralNetwork()
nn.load_models()

# Predicciones
preflop_bet = 0.05  
flop_bet = 0.1 
turn_bet = 0.2  

preflop_prediction = nn.preflop_model()
flop_prediction = nn.flop_model(preflop_bet)
turn_prediction = nn.turn_model(preflop_bet, flop_bet)
river_prediction = nn.river_model(preflop_bet, flop_bet, turn_bet)

print(f"Predicción de preflop: {preflop_prediction}")
print(f"Predicción de flop: {flop_prediction}")
print(f"Predicción de turn: {turn_prediction}")
print(f"Predicción de river: {river_prediction}")

Predicción de preflop: 0.23797049348869087
Predicción de flop: 0.22677880212421486
Predicción de turn: 0.17774877321739
Predicción de river: 0.845283752793023

Predicción de preflop: 0.23797049348869087
Predicción de flop: 0.22677880212421486
Predicción de turn: 0.17774877321739
Predicción de river: 0.845283752793023


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [31]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

class NeuralNetwork:
    def __init__(self):
        self.data = pd.read_csv('datasets/cleaned_data.csv')
        
        # Modelos para cada etapa
        self.preflop_mean = self.data['total'].mean()  # Media de lo que se suele apostar
        self.flop_predictor = None
        self.turn_predictor = None
        self.river_predictor = None
        
        self.load_data()
    
    def load_data(self):
        for stage in ['preflop', 'flop', 'turn', 'river']:
            self.data[[f'{stage}_player', f'{stage}_total']] = pd.DataFrame(self.data[stage].apply(eval).tolist(), index=self.data.index)
        self.data.drop(columns=['preflop', 'flop', 'turn', 'river'], inplace=True)
    
    def train_models(self) -> None:
        # Entrenar modelos con RandomForestRegressor
        self._train_flop_model()
        self._train_turn_model()
        self._train_river_model()
    
    def _train_flop_model(self) -> None:
        X = self.data[['preflop_total', 'preflop_player']]
        y = self.data['total']
        
        # Dividir datos con estrategia de randomización más robusta
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, 
            test_size=0.01,  # Aumentar tamaño de test set
            random_state=33
        )
        
        # Escalar características
        self.flop_scaler = StandardScaler()
        X_train_scaled = self.flop_scaler.fit_transform(X_train)
        
        # Entrenar modelo RandomForest
        self.flop_predictor = RandomForestRegressor(
            n_estimators=100,  # Más árboles
            max_depth=10,      # Profundidad de árboles
            min_samples_split=5,
            random_state=33
        )
        self.flop_predictor.fit(X_train_scaled, y_train)
    
    def _train_turn_model(self) -> None:
        X = self.data[['preflop_total', 'preflop_player', 'flop_total', 'flop_player']]
        y = self.data['total']
        
        # Dividir datos con estrategia de randomización más robusta
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, 
            test_size=0.01,  # Aumentar tamaño de test set
            random_state=33
        )
        
        # Escalar características
        self.turn_scaler = StandardScaler()
        X_train_scaled = self.turn_scaler.fit_transform(X_train)
        
        # Entrenar modelo RandomForest
        self.turn_predictor = RandomForestRegressor(
            n_estimators=150,  # Más árboles
            max_depth=12,      # Profundidad de árboles
            min_samples_split=5,
            random_state=33
        )
        self.turn_predictor.fit(X_train_scaled, y_train)
    
    def _train_river_model(self) -> None:
        X = self.data[['preflop_total', 'preflop_player', 'flop_total', 'flop_player', 
                       'turn_total', 'turn_player']]
        y = self.data['total']
        
        # Dividir datos con estrategia de randomización más robusta
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, 
            test_size=0.01,  # Aumentar tamaño de test set
            random_state=33  # Semilla aleatoria dinámica
        )
        
        # Escalar características
        self.river_scaler = StandardScaler()
        X_train_scaled = self.river_scaler.fit_transform(X_train)
        
        # Entrenar modelo RandomForest
        self.river_predictor = RandomForestRegressor(
            n_estimators=200,  # Más árboles
            max_depth=15,      # Profundidad de árboles
            min_samples_split=5,
            random_state=33
        )
        self.river_predictor.fit(X_train_scaled, y_train)
    
    def save_models(self, path_prefix: str = 'poker_model') -> None:
        # Guardar scaler
        joblib.dump(self.flop_scaler, f'{path_prefix}_flop_scaler.joblib')
        joblib.dump(self.turn_scaler, f'{path_prefix}_turn_scaler.joblib')
        joblib.dump(self.river_scaler, f'{path_prefix}_river_scaler.joblib')
        
        # Guardar modelos
        if self.flop_predictor:
            joblib.dump(self.flop_predictor, f'{path_prefix}_flop.joblib')
        if self.turn_predictor:
            joblib.dump(self.turn_predictor, f'{path_prefix}_turn.joblib')
        if self.river_predictor:
            joblib.dump(self.river_predictor, f'{path_prefix}_river.joblib')
        
        # Guardar media de preflop
        with open(f'{path_prefix}_preflop_mean.txt', 'w') as f:
            f.write(str(self.preflop_mean))
    
    def load_models(self, path_prefix: str = 'poker_model') -> None:
        # Cargar scaler
        self.flop_scaler = joblib.load(f'{path_prefix}_flop_scaler.joblib')
        self.turn_scaler = joblib.load(f'{path_prefix}_turn_scaler.joblib')
        self.river_scaler = joblib.load(f'{path_prefix}_river_scaler.joblib')
        
        # Cargar modelos
        try:
            self.flop_predictor = joblib.load(f'{path_prefix}_flop.joblib')
            self.turn_predictor = joblib.load(f'{path_prefix}_turn.joblib')
            self.river_predictor = joblib.load(f'{path_prefix}_river.joblib')
        except FileNotFoundError:
            print("Algunos modelos no se encontraron. Asegúrate de haberlos guardado previamente.")
        
        # Cargar media de preflop
        with open(f'{path_prefix}_preflop_mean.txt', 'r') as f:
            self.preflop_mean = float(f.read())
    
    def preflop_model(self) -> float:
        return self.preflop_mean
    
    def flop_model(self, preflop_bet: float) -> float:
        if self.flop_predictor is None:
            raise ValueError("El modelo de flop no ha sido entrenado. Entrena primero.")
        
        input_data = np.array([[preflop_bet, preflop_bet]])
        input_df = pd.DataFrame(input_data, columns=['preflop_total', 'preflop_player'])
        input_scaled = self.flop_scaler.transform(input_df[['preflop_total', 'preflop_player']])
        
        return float(self.flop_predictor.predict(input_scaled)[0])
    
    def turn_model(self, preflop_bet: float, flop_bet: float) -> float:
        if self.turn_predictor is None:
            raise ValueError("El modelo de turn no ha sido entrenado. Entrena primero.")
        
        input_data = np.array([[preflop_bet, preflop_bet, flop_bet, flop_bet]])
        input_df = pd.DataFrame(input_data, columns=['preflop_total', 'preflop_player', 'flop_total', 'flop_player'])
        input_scaled = self.turn_scaler.transform(input_df[['preflop_total', 'preflop_player', 'flop_total', 'flop_player']])
        
        return float(self.turn_predictor.predict(input_scaled)[0])
    
    def river_model(self, preflop_bet: float, flop_bet: float, turn_bet: float) -> float:
        if self.river_predictor is None:
            raise ValueError("El modelo de river no ha sido entrenado. Entrena primero.")
        
        input_data = np.array([[preflop_bet, preflop_bet, flop_bet, flop_bet, turn_bet, turn_bet]])
        input_df = pd.DataFrame(input_data, columns=['preflop_total', 'preflop_player', 'flop_total', 'flop_player', 'turn_total', 'turn_player'])
        input_scaled = self.river_scaler.transform(input_df[['preflop_total', 'preflop_player', 'flop_total', 'flop_player', 'turn_total', 'turn_player']])
        
        return float(self.river_predictor.predict(input_scaled)[0])

# Prueba 1
nn = NeuralNetwork()
nn.train_models()

# Predicciones
preflop_bet = 0.05 
flop_bet = 0.1  
turn_bet = 0.2  

preflop_prediction = nn.preflop_model()
flop_prediction = nn.flop_model(preflop_bet)
turn_prediction = nn.turn_model(preflop_bet, flop_bet)
river_prediction = nn.river_model(preflop_bet, flop_bet, turn_bet)

print(f"Predicción de preflop: {preflop_prediction}")
print(f"Predicción de flop: {flop_prediction}")
print(f"Predicción de turn: {turn_prediction}")
print(f"Predicción de river: {river_prediction}\n")

# Prueba tras guardar el modelo
nn.save_models()

nn = NeuralNetwork()
nn.load_models()

# Predicciones
preflop_bet = 0.05  
flop_bet = 0.1 
turn_bet = 0.2  

preflop_prediction = nn.preflop_model()
flop_prediction = nn.flop_model(preflop_bet)
turn_prediction = nn.turn_model(preflop_bet, flop_bet)
river_prediction = nn.river_model(preflop_bet, flop_bet, turn_bet)

print(f"Predicción de preflop: {preflop_prediction}")
print(f"Predicción de flop: {flop_prediction}")
print(f"Predicción de turn: {turn_prediction}")
print(f"Predicción de river: {river_prediction}")

Predicción de preflop: 0.23797049348869087
Predicción de flop: 0.0970691340092539
Predicción de turn: 0.3299598661526017
Predicción de river: 0.6113880487734492

Predicción de preflop: 0.23797049348869087
Predicción de flop: 0.0970691340092539
Predicción de turn: 0.3299598661526017
Predicción de river: 0.6113880487734492
